In [2]:
import pandas as pd
import numpy as np
import importlib
import xgboost
import Utils as util

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import pearsonr


In [4]:
df = pd.read_excel('Participants_Data_Used_Cars/Data_Train.xlsx')
tf = pd.read_excel('Participants_Data_Used_Cars/Data_Test.xlsx')


In [5]:
df = util.clean_data(df)
tf = util.clean_data(tf)
all_features = ['Brand', 'Model', 'Location', 'Owner_Type', 'Transmission', 'Fuel_Type', 'Engine', 'Power', 'Seats', 'Year', 'Kilometers_Driven']
numeric_features = ['Engine', 'Power', 'Seats', 'Year', 'Kilometers_Driven']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
    # ('scaler', StandardScaler())
    ])

categorical_features = ['Brand', 'Location', 'Owner_Type', 'Transmission', 'Fuel_Type']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('vect', CountVectorizer(), 'Model')
        # ('scale', StandardScaler(), all_features)
#        ('iter', IterativeImputer(max_iter=10, random_state=0), ['New_Price'])
        ])



In [6]:
X = df.drop(['Price', 'Location_Type', 'Name', 'Mileage','New_Price'], axis=1)
y = df['Price']
X_test = tf.drop(['Location_Type', 'Name'], axis=1)
all_data = X.append(X_test)
preprocessor.fit(all_data)


C:\Users\karthikn\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='most_frequent', verbose=0))]), ['Engine', 'Power', 'Seats', 'Year', 'Kilometers_Driven']), ('cat', Pipeline(memory=None,
     steps=[('imputer', SimpleIm...accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None), 'Model')])

In [7]:
X = preprocessor.transform(X)
X_test = preprocessor.transform(X_test)
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.1)

xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)


In [8]:
xgb.fit(X_train,y_train)
Y_pred = xgb.predict(X_validate)


C:\Users\karthikn\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:59:15] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [9]:
for i in range(len(Y_pred)):
   if Y_pred[i] < 0:
       Y_pred[i] = 0
#Printing the score for validation sets

y_true = y_validate
print("\n\n Linear Regression SCORE : ", util.score(Y_pred, y_true))




 Linear Regression SCORE :  0.9282040306396078


In [10]:
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)


In [11]:
xgb.fit(X,y)
Y_test = xgb.predict(X_test)


C:\Users\karthikn\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[16:07:44] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [12]:
result_df = pd.DataFrame({'Price':Y_test})
## save to xlsx file

filepath = 'results_1.xlsx'
result_df.to_excel(filepath, index=False)
